<a href="https://colab.research.google.com/github/Saumya0330/Machine_Learning_Saumya_UML501/blob/main/ML_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab Assignment 3

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Q1: K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit)

Load the dataset and Implement 5- fold cross validation for multiple linear regression
(using least square error fit).

a) Divide the dataset into input features (all columns except price) and output variable  
(price)

In [3]:
data=pd.read_csv("/content/USA_Housing.csv")
data.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [4]:
x=data.drop('Price',axis=1) #input features
y=data['Price'] #label

In [5]:
data.shape

(5000, 6)

b) Scale the values of input features.

In [52]:
class MinMaxScaler():
  def __init__(self):
    self.min=None
    self.max=None

  def fit(self,x):
    self.min=np.min(x,axis=0)
    self.max=np.max(x,axis=0)

  def transform(self,x):
    return (x-self.min)/(self.max-self.min)

  def fit_transform(self,x):
    self.fit(x)
    return self.transform(x)

In [7]:
scaler=MinMaxScaler()
x_scaled=scaler.fit_transform(x)
x_scaled.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,0.686822,0.441986,0.501502,0.464444,0.329942
1,0.683521,0.488538,0.464501,0.242222,0.575968
2,0.483737,0.468609,0.701350,0.695556,0.528582
3,0.506630,0.660956,0.312430,0.280000,0.491549
4,0.469223,0.348556,0.611851,0.495556,0.376988


c) Divide input and output features into five folds.

In [8]:
def kfold(n_folds, x, y):
  x_folds = np.array_split(x, n_folds, axis=0)
  y_folds = np.array_split(y, n_folds, axis=0)
  return x_folds, y_folds

In [9]:
x_folds, y_folds = kfold(5, x_scaled.to_numpy(), y.to_numpy())

In [10]:
x_folds[0].shape

(1000, 5)

d) Run five iterations, in each iteration consider one-fold as test set and remaining four sets as training set. Find the beta (𝛽) matrix, predicted values, and R2_score

In [11]:
def r2_score(y_test, y_pred):
    sse = np.sum((y_test-y_pred)**2)
    sst = np.sum((y_test-np.mean(y_test))**2)
    r2 = 1- sse/sst
    return r2

In [12]:
def mlr(x_train, y_train, x_test, y_test):
  x_b = np.hstack((np.ones((x_train.shape[0], 1)),x_train)) #adding ones column for getting intercept B0
  # print(x_b.shape)
  # print(y_train.shape)

  beta = np.linalg.inv(x_b.T.dot(x_b)).dot(x_b.T).dot(y_train)
  #print(beta.ravel()) #the beta coefficients for mlr equation
  x_test = np.hstack((np.ones((x_test.shape[0],1)), x_test))
  y_pred = x_test.dot(beta)

  return beta, y_pred, r2_score(y_test, y_pred)


In [25]:
def cross_validate(x_folds, y_folds):
  results=[]
  for i in range(len(x_folds)):

    x_test = x_folds[i]
    y_test = y_folds[i]
    x_train = np.concatenate(x_folds[:i]+x_folds[i+1:])
    y_train = np.concatenate(y_folds[:i]+y_folds[i+1:])

    beta, y_pred, r2 = mlr(x_train, y_train,x_test, y_test)
    # print('beta matrix: ',beta)
    # print('predicted values: ', y_pred[:10])
    # print('r2 score: ', r2,'\n\n') #closer to 1 shows a good fit

    results.append((beta,r2))

  return results

In [26]:
results1 = cross_validate(x_folds,y_folds) # this output is only for the 1st fold
print(results1)


[(array([-1422826.49058154,  1939707.44686449,  1137661.5980838 ,
         904570.15373278,    12886.39203736,  1062629.53134166]), np.float64(0.9175899480765111)), (array([-1415097.52771118,  1926560.46870721,  1145623.78774051,
         916657.86906761,     6188.46744136,  1050035.41463358]), np.float64(0.9203015496401122)), (array([-1423350.92494848,  1946511.37171666,  1134205.13423584,
         910146.97109357,     9555.10727824,  1056104.51551669]), np.float64(0.915242991532002)), (array([-1414578.4117767 ,  1939689.35012527,  1140921.38723211,
         895697.76568006,     4099.75517048,  1058866.3235135 ]), np.float64(0.9208503836977662)), (array([-1418234.68515569,  1946919.614116  ,  1135596.09355141,
         911763.12050533,     4547.40335147,  1050234.71063083]), np.float64(0.9138111758717493))]


e) Use the best value of (𝛽) matrix (for which R2_score is maximum), to train the regressor for 70% of data, and test the data for remaining 30% data

In [27]:
def test_train_split(x_scaled, y, test_size=0.3):
  x_train = x_scaled[:int(x_scaled.shape[0]*(1-test_size))]
  y_train = y[:int(y.shape[0]*(1-test_size))]
  x_test = x_scaled[int(x_scaled.shape[0]*test_size):]
  y_test = y[int(y.shape[0]*test_size):]

  return x_train, y_train, x_test, y_test

In [29]:
results = cross_validate(x_folds,y_folds)
best_beta, best_r2 = max(results, key=lambda x: x[1])
# print(best_beta)
# print(best_r2)
x_train, y_train, x_test, y_test = test_train_split(x_scaled, y, 0.3)
beta_final, y_pred_final, r2_final=mlr(x_train, y_train, x_test, y_test)

print('beta matrix: ',beta_final)
print('predicted values: ', y_pred_final[:10])
print('r2 score: ', r2_final)

beta matrix:  [-1418134.51980246  1949011.02385071  1138170.74985997   906046.25782495
     4779.74654042  1051598.12106512]
predicted values:  [1318343.84522536 1310435.53697415 1043614.98210099 1312966.11930088
 1263691.48581127 1545982.38007362 1237266.23474307  771005.7709474
 1117138.28907133 1686300.23505073]
r2 score:  0.9180539196369379


 2. Concept of Validation set for Multiple Linear Regression (Gradient Descent Optimization)  

Consider the same dataset of Q1, rather than dividing the dataset into five folds, divide the
dataset into training set (56%), validation set (14%), and test set (30%).  
Consider four different values of learning rate i.e. {0.001,0.01,0.1,1}. Compute the values of
regression coefficients for each value of learning rate after 1000 iterations.  
For each set of regression coefficients, compute R2_score for validation and test set and find the best value of regression coefficients

In [30]:
def train_valid_test_split(x, y, test_size=0.3, valid_size=0.14):
  split2 = int(x.shape[0]*(1-test_size))
  split1 = int(x.shape[0]*(1-(test_size+valid_size)))

  x_train, x_valid, x_test = x[:split1], x[split1:split2], x[split2:]
  y_train, y_valid, y_test = y[:split1], y[split1:split2], y[split2:]

  return x_train, y_train, x_valid, y_valid, x_test, y_test

In [37]:
def loss_func(beta, x, y):
  n = x.shape[0]
  return (1/n)*((x.dot(beta)-y).T)*((x.dot(beta)-y))

def loss_func_derivative(beta, x, y):
  n = x.shape[0]
  return (2/n)*(x.T.dot((x.dot(beta)-y)))

def gradient_descent(beta, x, y, n_iter, eta):
  for i in range(n_iter):
    beta = beta - eta*loss_func_derivative(beta,x,y)
  return beta

def mlr(x_train, y_train, x_valid, y_valid, x_test, y_test, learning_rate):
  x_b_train= np.hstack((np.ones((x_train.shape[0],1)),x_train))
  x_b_valid = np.hstack((np.ones((x_valid.shape[0],1)),x_valid))
  x_b_test = np.hstack((np.ones((x_test.shape[0],1)),x_test))

  y_train = y_train.to_numpy().reshape(-1,1)
  y_valid = y_valid.to_numpy().reshape(-1,1)
  y_test = y_test.to_numpy().reshape(-1,1)

  #input initial beta matrix
  beta_initial = np.zeros((x_b_train.shape[1],1))
  results=[]
  for eta in learning_rate:

    beta_final = gradient_descent(beta_initial, x_b_train, y_train, 1000, eta)
    y_pred_valid = x_b_valid.dot(beta_final)
    y_pred_test = x_b_test.dot(beta_final)

    r2_valid = r2_score(y_valid, y_pred_valid)
    r2_test = r2_score(y_test, y_pred_test)
    results.append((beta_final, r2_valid, r2_test))
  return results


In [38]:
x_train, y_train, x_valid, y_valid, x_pred, y_pred = train_valid_test_split(x_scaled, y, 0.3, 0.14)
learning_rate=[0.001,0.01,0.1,1]
results = mlr(x_train, y_train, x_valid, y_valid, x_pred, y_pred, learning_rate)

best_beta, best_r2_valid, best_r2_test = max(results, key=lambda x: (x[1],x[2]))

print('beta matrix: ',best_beta)
print('r2 score for validation set: ', best_r2_valid)
print('r2 score for test set: ', best_r2_test)

beta matrix:  [[-1004477.68037321]
 [ 1641352.31612067]
 [  999179.41619036]
 [  686100.59731705]
 [   33761.8011126 ]
 [  910718.45921662]]
r2 score for validation set:  0.8914022789697157
r2 score for test set:  0.895864348159036


Pre-processing and Multiple Linear Regression  

1. Load the dataset with following column names ["symboling", "normalized_losses",  
"make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels",  
"engine_location", "wheel_base", "length", "width", "height", "curb_weight",  
"engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",  
"compression_ratio", "horsepower", "peak_rpm","city_mpg","highway_mpg","price"] and replace all ? values with nan


In [54]:
column_names = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels","engine_location", "wheel_base", "length", "width", "height", "curb_weight","engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke","compression_ratio", "horsepower", "peak_rpm","city_mpg","highway_mpg","price"]
data1 = pd.read_csv("/content/imports-85_data.csv", names=column_names, header=None, na_values="?")
data1.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


2. Replace all NaN values with central tendency imputation. Drop the rows with NaN values in price column.

In [5]:
data1.isna().sum()

,0
symboling,0
normalized_losses,41
make,0
fuel_type,0
aspiration,0
num_doors,2
body_style,0
drive_wheels,0
engine_location,0
wheel_base,0


In [53]:
data1.shape

(205, 26)

In [55]:
data1 = data1.dropna(subset=["price"])

# Select only numeric columns for calculating the mean
numeric_cols = data1.select_dtypes(include=np.number)

# Fill NaN values in the original DataFrame with the mean of corresponding numeric columns
data1[numeric_cols.columns] = data1[numeric_cols.columns].fillna(numeric_cols.mean())

In [12]:
data1.isna().sum()

,0
symboling,0
normalized_losses,0
make,0
fuel_type,0
aspiration,0
num_doors,2
body_style,0
drive_wheels,0
engine_location,0
wheel_base,0


In [7]:
data1.shape

(201, 26)

3. There are 10 columns in the dataset with non-numeric values. Convert these values to  
numeric values using following scheme:  
(i) For “num_doors” and “num_cylinders”: convert words (number names) to figures  
for e.g., two to 2  
(ii) For "body_style", "drive_wheels": use dummy encoding scheme  
(iii) For “make”, “aspiration”, “engine_location”,fuel_type: use label encoding  
scheme  
(iv) For fuel_system: replace values containing string pfi to 1 else all values to 0.  

In [56]:
print(data1['num_doors'].unique(), data1['num_cylinders'].unique())
num_map={
    'two':2,
    'three':3,
    'four':4,
    'five':5,
    'six':6,
    'eight':8,
    'twelve':12
}
data1['num_doors']=data1['num_doors'].map(num_map)
data1['num_cylinders']=data1['num_cylinders'].map(num_map)


['two' 'four' nan] ['four' 'six' 'five' 'three' 'twelve' 'two' 'eight']


In [72]:
data1['num_doors']=data1['num_doors'].fillna(data1['num_doors'].mode()[0])

In [58]:
print("Columns before get_dummies:", data1.columns.tolist())

data1 = pd.get_dummies(data1, columns=['body_style', 'drive_wheels'], dtype=int)

print("\nColumns after get_dummies:", data1.columns.tolist())
print("\nHead of the DataFrame after get_dummies:")
print(data1.head())

Columns before get_dummies: ['symboling', 'normalized_losses', 'make', 'fuel_type', 'aspiration', 'num_doors', 'body_style', 'drive_wheels', 'engine_location', 'wheel_base', 'length', 'width', 'height', 'curb_weight', 'engine_type', 'num_cylinders', 'engine_size', 'fuel_system', 'bore', 'stroke', 'compression_ratio', 'horsepower', 'peak_rpm', 'city_mpg', 'highway_mpg', 'price']

Columns after get_dummies: ['symboling', 'normalized_losses', 'make', 'fuel_type', 'aspiration', 'num_doors', 'engine_location', 'wheel_base', 'length', 'width', 'height', 'curb_weight', 'engine_type', 'num_cylinders', 'engine_size', 'fuel_system', 'bore', 'stroke', 'compression_ratio', 'horsepower', 'peak_rpm', 'city_mpg', 'highway_mpg', 'price', 'body_style_convertible', 'body_style_hardtop', 'body_style_hatchback', 'body_style_sedan', 'body_style_wagon', 'drive_wheels_4wd', 'drive_wheels_fwd', 'drive_wheels_rwd']

Head of the DataFrame after get_dummies:
   symboling  normalized_losses         make fuel_type

In [59]:
from sklearn.preprocessing import LabelEncoder

label_cols = ["make", "aspiration", "engine_location", "fuel_type"]

encoder = LabelEncoder()
for col in label_cols:
    data1[col] = encoder.fit_transform(data1[col])

print(data1.head())

   symboling  normalized_losses  make  fuel_type  aspiration  num_doors  \
0          3              122.0     0          1           0        2.0   
1          3              122.0     0          1           0        2.0   
2          1              122.0     0          1           0        2.0   
3          2              164.0     1          1           0        4.0   
4          2              164.0     1          1           0        4.0   

   engine_location  wheel_base  length  width  ...  highway_mpg    price  \
0                0        88.6   168.8   64.1  ...           27  13495.0   
1                0        88.6   168.8   64.1  ...           27  16500.0   
2                0        94.5   171.2   65.5  ...           26  16500.0   
3                0        99.8   176.6   66.2  ...           30  13950.0   
4                0        99.4   176.6   66.4  ...           22  17450.0   

  body_style_convertible  body_style_hardtop  body_style_hatchback  \
0                     

In [60]:
data1["fuel_system"] = data1["fuel_system"].apply(lambda x: 1 if "pfi" in x.lower() else 0)


In [73]:
data1.isna().sum()

,0
symboling,0
normalized_losses,0
make,0
fuel_type,0
aspiration,0
num_doors,0
engine_location,0
wheel_base,0
length,0
width,0


4. Divide the dataset into input features (all columns except price) and output variable (price). Scale all input features

In [75]:
# Select only numeric columns for input features
numeric_cols_x = data1.select_dtypes(include=np.number).drop('price', axis=1)
x = numeric_cols_x
y = data1['price']

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

 5. Train a linear regressor on 70% of data (using inbuilt linear regression function of Python). Test its performance on remaining 30% of data.


In [63]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [76]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
# Predictions
y_pred = model.predict(X_test)

# Metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R² Score:", r2)

Mean Squared Error: 12539653.421596656
R² Score: 0.8677135345531459


6. Reduce the dimensionality of the feature set using inbuilt PCA decomposition and then again train a linear regressor on 70% of reduced data (using inbuilt linear regression function of Python). Does it lead to any performance improvement on test set?

In [78]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [80]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

pca = PCA(n_components=0.95)   # keep 95% variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca  = pca.transform(X_test_scaled)

print("Original shape:", X_train.shape)
print("Reduced shape:", X_train_pca.shape)


Original shape: (140, 30)
Reduced shape: (140, 16)


In [82]:
model_pca = LinearRegression()
model_pca.fit(X_train_pca, y_train)

y_pred_pca = model_pca.predict(X_test_pca)

mse_pca = mean_squared_error(y_test, y_pred_pca)
r2_pca = r2_score(y_test, y_pred_pca)

print("PCA-based Model -> MSE:", mse_pca, " R²:", r2_pca)

PCA-based Model -> MSE: 12777361.962891566  R²: 0.8652058398284836


In [83]:
print("Original Model -> MSE:", mse, " R²:", r2)
print("PCA Model      -> MSE:", mse_pca, " R²:", r2_pca)


Original Model -> MSE: 12539653.421596656  R²: 0.8677135345531459
PCA Model      -> MSE: 12777361.962891566  R²: 0.8652058398284836


features already had useful information, and PCA lost some.